# **Extraer con selenium datos de licitacións do Concello da Coruña** 

**SQL Server**

- docker run -e "ACCEPT_EULA=Y" -e "MSSQL_SA_PASSWORD=abc123." -e "MSSQL_PID=Evaluation" -p 41433:1433  --name sqlpreview --hostname sqlpreview -d mcr.microsoft.com/mssql/server:2022-preview-ubuntu-22.04

In [257]:
#Instalar gecko
from webdriver_manager.firefox import GeckoDriverManager
#GeckoDriverManager().install()

#Abrir un navegador
from selenium import webdriver
from selenium.webdriver.common.by import By
driver = webdriver.Firefox()

In [258]:
import argparse
from enum import Enum
import sys
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select
import time
import random


In [297]:
def waitFinishLoad(driver,idLoader,needsDisapeared=False,by=By.ID,needsWait=False):
    if idLoader is None:
        return
    WebDriverWait(driver, timeout=10).until(EC.presence_of_element_located((by, idLoader)))
    if needsDisapeared:
        WebDriverWait(driver, timeout=10).until(EC.invisibility_of_element((by, idLoader)))
    if needsWait:
        time.sleep(random.choice([0.1,0.2,0.3,0.4,0.5,0.6]))

def elementBy(query,driver,idLoader=None,by=By.ID,notStart=False,needsWait=False):
    if not notStart:
        waitFinishLoad(driver,idLoader,by=by,needsWait=needsWait)
    element=driver.find_element(By.XPATH,query)
    waitFinishLoad(driver,idLoader,by=by,needsWait=needsWait)
    return element

def elementByClick(query,driver,idLoader=None,by=By.ID,notStart=False,needsWait=False):
    if not notStart:
        waitFinishLoad(driver,idLoader,by=by,needsWait=needsWait)
    element=driver.find_element(By.XPATH,query)
    element.click()
    waitFinishLoad(driver,idLoader,by=by,needsWait=needsWait)
    return element

def elementsBy(query,driver,idLoader=None,by=By.ID,notStart=False,needsWait=False):
    if notStart:
        waitFinishLoad(driver,idLoader,by=by,needsWait=needsWait)
    element=driver.find_elements(By.XPATH,query)
    waitFinishLoad(driver,idLoader,by=by,needsWait=needsWait)
    return element

def clickEvent(driver,element):
    driver.execute_script(element.get_dom_attribute("onClick"))

def goToNextPage(driver):
    elementByClick("//input[@value='Siguiente']",driver,idLoader="//table[@id='myTablaBusquedaCustom']",by=By.XPATH)

Entramos en la página y nos vamos a publicaciones y luego licitaciones

In [260]:
driver.get("https://contrataciondelestado.es")

In [261]:
elementBy("//a[@title='Buscar publicaciones']",driver,"footer-newShow").click()

In [262]:
# No es un enlace cualquiera, este esta gestionado por un script que controlaremos por código 
licitacion=elementBy("//div[@class='divLogo']/a",driver,"footer-newShow")

clickEvent(driver,licitacion)

In [263]:
busqueda_avanzada=elementBy("//div[@class='capaAvanzada']/a",driver,"contenidoBuscador")

clickEvent(driver,busqueda_avanzada)

In [264]:
seleccionar=elementBy("//div[@class='inlinebloque   ']/a",driver,by=By.XPATH,idLoader="//div[@class='inlinebloque   ']/a")
clickEvent(driver,seleccionar)

In [265]:
def get_corunha(list,driver,i=2):
    """_summary_

    Args:
        list (_type_): _description_
        driver (_type_): _description_
        i (int, optional): _description_. Defaults to 2.
    """
    if i-2<len(list):
        q=f"//*[text()='{list[i-2]}']"
        if len(list)!=i-1:
            q+="/../.."
            query=f"{q}/td[{i}]"
            total = f"{q}/../../../.." if i!=2 else 'footer-newShow'
        else:
            query=q
            q2=f"{q}/../.."
            total = f"{q2}/../../../.."
        if len(list)!=i-1:
            elementByClick(query,driver,by=By.XPATH if i!=2 else By.ID,idLoader=total,notStart=True)
        else:
            return elementsBy(query,driver,by=By.XPATH if i!=2 else By.ID,idLoader=total,notStart=True)
        if i-1!=len(list):
            return get_corunha(list,driver,i+1)

In [266]:
#query="//*[text()='ENTIDADES LOCALES']/../../td[2]"
#elemento=elementByXPATH(query,driver,"footer-newShow")
#elemento.click()
waitFinishLoad(driver,by=By.XPATH,idLoader="//*[text()='ENTIDADES LOCALES']/../../td[2]")
get_corunha(['ENTIDADES LOCALES','Galicia','A Coruña','Ayuntamientos','A Coruña'],driver)[1].click()
elementByClick("//*[@value='Añadir']",driver)




<selenium.webdriver.remote.webelement.WebElement (session="d4452e5b-9737-4b15-ad6f-570bb3e93956", element="9f8e2000-025b-4066-8257-0627adc45989")>

In [267]:
elementByClick("//*[@title='Buscar']",driver)

<selenium.webdriver.remote.webelement.WebElement (session="d4452e5b-9737-4b15-ad6f-570bb3e93956", element="27399b5e-7f3c-43af-832f-c2998552fe06")>

In [313]:
from bs4 import BeautifulSoup
import pandas as pd


waitFinishLoad(driver,by=By.XPATH,idLoader="//div/span[text()='Importe']")
#df=pd.read_html(elementBy("//table[@id='myTablaBusquedaCustom']",driver).get_attribute('outerHTML'))[0]
soup=BeautifulSoup(driver.page_source,'html.parser')
df=pd.read_html(str(soup.find(id="myTablaBusquedaCustom")))[0]
df[~df.iloc[:,0].str.startswith('Página')]

,Expediente,Tipo de Contrato,Estado,Importe,Fechas,Órgano de Contratación
0,2023/SP03032005/00009996Contrato de servicio d...,ServiciosServicios de limpieza de edificios y ...,Adjudicada,"10.680,00",Publicación PLACSP:Adjudicación:05/05/2023,Subdirección General de Gestión Económica
1,114/23Servicio de seguros para INCIBE,ServiciosServicios financieros: a) servicios d...,Parcialmente Resuelta,"63.480,00",Publicación PLACSP:Adjudicación:22/01/2024Form...,Dirección General del Instituto Nacional de Ci...
2,CPP002/22Documento Regulador de Compra Pública...,ServiciosServicios de investigación y desarrollo,Resuelta,"137.200.000,00",Publicación PLACSP:Adjudicación:16/06/2023Form...,Dirección General del Instituto Nacional de Ci...
3,UNED BARBASTRO PNSP 1/2023Suministro de embala...,SuministrosAdquisición,Resuelta,"32.065,00",Publicación PLACSP:Adjudicación:01/02/2024,Consorcio Universitario del Centro Asociado a ...
4,UNED BARBASTRO PAS 2/2024Servicio de asesoría ...,ServiciosServicios jurídicos,Publicada,"12.500,00",Present. Oferta:21/02/2024,Consorcio Universitario del Centro Asociado a ...
5,05/2024Servicio de actividades de envejecimien...,ServiciosOtros servicios,Publicada,"73.500,00",Present. Oferta:21/02/2024,Delegación Municipal de Hacienda
6,6/2024Servicio de soporte de la aplicación act...,ServiciosServicios de mantenimiento y reparación,Publicada,"9.600,00",Present. Oferta:21/02/2024,Comisión Ejecutiva de la Gerencia de Urbanismo...
7,245/2024Lic 1/245/2024 - Servicios de Sobretit...,ServiciosOtros servicios,Publicada,"45.000,00",Present. Oferta:20/02/2024,Gerencia del Auditorio de Tenerife SAU
8,2023/PL110La Entidad precisa formalizar la Con...,ObrasConstrucción,Adjudicada,"2.190.612,32",Publicación PLACSP:Adjudicación:31/01/2024,Consejo de Administración de Limpieza de Málag...
9,CEA27/2024Componentes De Seguridad En Red Para...,SuministrosAdquisición,Adjudicada,"136.329,02",Publicación PLACSP:Adjudicación:06/02/2024,Dirección General de Racionalización y Central...


In [298]:
goToNextPage(driver)